In [2]:
from __future__ import annotations
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../src/")

from utils import *
from sklearn.impute import KNNImputer
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import geopandas as gpd

In [2]:
#appel d'une fonction dans utils pour ouvrir le dataset des accidents et le formatter
df_france = create_dataframe_france()

c:\Users\Pierr\OneDrive\Documents\CentraleSupelec\ChallengeIA\challenge_ia_accidentologie_velos\notebook\../src\utils.py:163: DtypeWarning: Columns (11,14,19,20,24,25,34,40,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("..", "data", "dataset_velo_acc_preprocess.csv"))


In [3]:
df_france.head()

,index,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,...,com,adr,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg
0,26,1,0.0,3.0,0.0,2.0,0.0,9,2.0,NaN,...,121.0,"., FRANCE ( AVE ANATOLE",47.74613,-3.36700,560,0.0,2.0,0.0,2010-12-11,1
1,33,2,0.0,3.0,0.0,2.0,3.0,3,NaN,NaN,...,94.0,rue de tanouarn,48.35744,-1.71484,350,NaN,NaN,NaN,2011-01-19,1
2,34,2,0.0,3.0,0.0,2.0,6.0,3,NaN,NaN,...,193.0,NaN,48.23972,-1.71215,350,0.0,2.0,7.0,2011-02-28,0
3,35,2,0.0,3.0,0.0,2.0,1.0,4,NaN,NaN,...,584.0,rue v.hugo/rue debussy,43.86300,1.49563,310,0.0,2.0,3.0,2011-04-20,1
4,37,1,0.0,3.0,0.0,9.0,0.0,4,NaN,NaN,...,738.0,NaN,49.65811,3.31329,20,0.0,2.0,0.0,2011-04-21,1


In [12]:
df_amenagement = gpd.read_file("../data/france-20230101.geojson")

In [13]:
df_amenagement.loc[df_amenagement["ame_g"]=="AUCUN"].head()

,id_local,id_osm,num_iti,code_com_d,ame_d,regime_d,sens_d,largeur_d,local_d,statut_d,...,revet_g,access_ame,date_maj,trafic_vit,lumiere,d_service,source,project_c,ref_geo,geometry
14,geovelo_90581450_16015,90581450,None,16015,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,...,LISSE,None,2022-10-22,50.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.17274 45.64648, 0.17263 45.64651..."
16,geovelo_492304837_72181,492304837,None,72181,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2020-04-27,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.16681 47.97614, 0.16603 47.97593..."
20,geovelo_1079978026_72181,1079978026,None,72181,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2022-07-20,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.18191 48.01231, 0.18181 48.01235)"
22,geovelo_574123079_72008,574123079,None,72008,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2018-03-28,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.18683 47.93192, 0.18634 47.93083..."
24,geovelo_851613127_72181,851613127,None,72181,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,...,LISSE,None,2020-09-25,50.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.20236 48.01569, 0.20232 48.01570..."


In [14]:
df_amenagement = preprocessing(df_amenagement)

In [15]:
df_amenagement["code_com"] = df_amenagement["code_com"].astype("string")
df_amenagement = df_amenagement.loc[df_amenagement["code_com"].str[:2] == "75"]
df_amenagement

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
403,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513
407,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210
408,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913
409,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907
425,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114
...,...,...,...,...,...,...,...,...
255679,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756
255681,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300
255686,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840
255689,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053


In [16]:
df_amenagement.reset_index(inplace = True)
df_amenagement

,index,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,403,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513
1,407,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210
2,408,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913
3,409,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907
4,425,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114
...,...,...,...,...,...,...,...,...,...
9694,255679,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756
9695,255681,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300
9696,255686,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840
9697,255689,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053


In [17]:
np.array(df_paris.index)

array([   0,    1,    2, ..., 3174, 3175, 3176], dtype=int64)

In [18]:
df_amenagement["date_maj"] = pd.to_datetime(df_amenagement["date_maj"])

In [19]:
infrastructure = df_amenagement[['latitude_dep', 'longitude_dep']].to_numpy()
accidents = df_paris[['lat', 'long']].to_numpy()

In [20]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians, asin
from tqdm import tqdm
import warnings

warnings.simplefilter('ignore')

def distance(lat_acc, lon_acc, lat_fin, lon_fin, lat_dep, lon_dep):
    R = 6371
    lat_dep, lon_dep, lat_acc, lon_acc = map(np.radians, [lat_dep, lon_dep, lat_acc, lon_acc])
    
    dlat = lat_acc - lat_dep
    dlon = lon_acc - lon_dep
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_dep) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance1 = R * c
    
    lat_fin, lon_fin = map(np.radians, [lat_fin, lon_fin])
    
    dlat = lat_acc - lat_fin
    dlon = lon_acc - lon_fin
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_fin) * np.cos(lat_acc) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance2 = R * c

    if(distance1.min()>distance2.min()):
        return distance2
    else:
        return distance1

def link_dataframes(accidents, infrastructure, radius):
    # On ajoute une colonne distance qui sera utilisé pour la jointure
    accidents['distance'] = np.nan
    accidents['Index_in_accident'] = np.nan
    for i, row in tqdm(accidents.iterrows(), total=accidents.shape[0]):
        lat1, lon1 = row['lat'], row['long']
        
        infrastructure['distance'] = distance(lat1, lon1,
                                              infrastructure['latitude_fin'],
                                              infrastructure['longitude_fin'],
                                              infrastructure['latitude_dep'],
                                              infrastructure['longitude_dep'])
        distance_series = infrastructure.query("distance <= @radius").distance
        accidents.at[i, 'distance'] = distance_series.min()*1000
        accidents.at[i, 'Index_in_accident'] = distance_series.argmin()
    
    infrastructure.drop(["index", "distance"], inplace=True, axis=1)
    accidents.drop(["index"], inplace=True, axis=1)
    
    return accidents

#On utilise les dataframes accidents et infrastructure
infrastructure_with_acc = link_dataframes(df_paris, df_amenagement, 500)

100%|██████████| 3177/3177 [00:49<00:00, 63.82it/s]


In [21]:
infrastructure_with_acc

,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,...,lat,long,dep,obs2,obsm2,choc2,date,is_in_agg,distance,Index_in_accident
0,1,0.0,1.0,0.0,0.0,5.0,4,1.0,0.0,1.0,...,48.834640,2.438930,750,0.0,0.0,3.0,2010-10-03,1,75.688441,9685.0
1,1,0.0,2.0,0.0,2.0,0.0,4,2.0,0.0,0.0,...,48.525010,2.214320,750,0.0,2.0,0.0,2015-08-13,1,33597.765098,3234.0
2,1,0.0,3.0,0.0,2.0,2.0,4,1.0,1.0,0.0,...,48.531810,2.210700,750,0.0,0.0,7.0,2015-06-27,1,32938.334476,3234.0
3,3,0.0,2.0,0.0,2.0,8.0,4,1.0,1.0,0.0,...,48.880770,2.309110,750,0.0,2.0,1.0,2015-10-08,1,39.504248,3993.0
4,1,1.0,1.0,0.0,2.0,0.0,4,1.0,4.0,1.0,...,48.859300,2.348660,750,0.0,2.0,0.0,2016-03-23,1,19.988341,9209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,1,1.0,3.0,0.0,2.0,0.0,4,2.0,2.0,1.0,...,48.877110,2.326913,75,0.0,2.0,0.0,2021-01-13,1,0.234017,9029.0
3173,1,1.0,4.0,0.0,2.0,8.0,4,2.0,2.0,0.0,...,48.882039,2.301056,75,0.0,2.0,2.0,2021-01-11,1,150.279165,4488.0
3174,1,2.0,3.0,0.0,2.0,1.0,4,1.0,3.0,1.0,...,48.883869,2.368592,75,0.0,2.0,6.0,2021-01-09,1,2.007216,5740.0
3175,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,0.0,...,48.848926,2.380025,75,0.0,2.0,3.0,2021-01-07,1,83.304940,7157.0


In [22]:
infrastructure_with_acc.loc[infrastructure_with_acc['distance']<50]['distance'].describe()

count    2339.000000
mean       12.442419
std        13.423714
min         0.011119
25%         2.441165
50%         6.170638
75%        19.487049
max        49.959799
Name: distance, dtype: float64

In [23]:
infrastructure_with_acc['Index_in_accident'] = infrastructure_with_acc['Index_in_accident'].astype("Int32")

In [24]:
df_amenagement['Index_in_accident'] = df_amenagement.index
df_amenagement

,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin,Index_in_accident
0,75116,BANDE CYCLABLE,EN AGGLOMERATION,2022-11-04,48.861393,2.267261,48.861632,2.267513,0
1,75115,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2020-12-23,48.832573,2.279247,48.832508,2.279210,1
2,75115,PISTE CYCLABLE,AUTRE,2022-11-02,48.832709,2.278863,48.832772,2.278913,2
3,75116,AUTRE,AUTRE,2022-05-18,48.857419,2.280977,48.857452,2.280907,3
4,75117,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2022-04-16,48.875351,2.296177,48.875369,2.296114,4
...,...,...,...,...,...,...,...,...,...
9694,75112,AUTRE,AUTRE,2022-11-30,48.843625,2.456756,48.843625,2.456756,9694
9695,75112,BANDE CYCLABLE,ZONE 30,2022-06-28,48.818295,2.461300,48.818295,2.461300,9695
9696,75112,PISTE CYCLABLE,AUTRE,2018-07-09,48.826301,2.464644,48.825301,2.464840,9696
9697,75112,PISTE CYCLABLE,AUTRE,2022-11-02,48.842247,2.463053,48.842247,2.463053,9697


In [25]:
infrastructure_with_acc = infrastructure_with_acc.merge(df_amenagement, left_on="Index_in_accident", right_on="Index_in_accident")


,grav,senc,catv,obs,obsm,choc,catr,circ,nbv,vosp,...,distance,Index_in_accident,code_com,ame,regime,date_maj,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,1,0.0,1.0,0.0,0.0,5.0,4,1.0,0.0,1.0,...,75.688441,9685,75112,AUTRE,AUTRE,2022-04-22,48.834471,2.437928,48.834569,2.437893
1,1,0.0,2.0,0.0,2.0,0.0,4,2.0,0.0,0.0,...,33597.765098,3234,75114,BANDE CYCLABLE,EN AGGLOMERATION,2022-10-12,48.817002,2.331978,48.817004,2.331960
2,1,0.0,3.0,0.0,2.0,2.0,4,1.0,1.0,0.0,...,32938.334476,3234,75114,BANDE CYCLABLE,EN AGGLOMERATION,2022-10-12,48.817002,2.331978,48.817004,2.331960
3,1,1.0,3.0,0.0,2.0,5.0,4,1.0,1.0,0.0,...,35292.801953,3234,75114,BANDE CYCLABLE,EN AGGLOMERATION,2022-10-12,48.817002,2.331978,48.817004,2.331960
4,3,0.0,2.0,0.0,2.0,8.0,4,1.0,1.0,0.0,...,39.504248,3993,75108,AUTRE,EN AGGLOMERATION,2022-02-21,48.880421,2.309208,48.880440,2.309341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,1,3.0,1.0,0.0,2.0,8.0,4,1.0,2.0,0.0,...,19.699469,5391,75120,PISTE CYCLABLE,AUTRE,2022-10-13,48.866683,2.412775,48.866667,2.412660
3173,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,2.0,...,39.391036,5553,75114,DOUBLE SENS CYCLABLE NON MATERIALISE,ZONE 30,2021-04-30,48.824748,2.331264,48.824773,2.331165
3174,1,1.0,3.0,0.0,2.0,0.0,4,2.0,2.0,1.0,...,0.234017,9029,75108,DOUBLE SENS CYCLABLE BANDE,EN AGGLOMERATION,2022-09-14,48.877110,2.326916,48.877077,2.326915
3175,1,1.0,3.0,0.0,2.0,1.0,4,2.0,2.0,0.0,...,83.304940,7157,75112,DOUBLE SENS CYCLABLE NON MATERIALISE,AIRE PIETONNE,2022-03-03,48.849294,2.381067,48.849298,2.381013


In [ ]:
infrastructure_with_acc

In [27]:
infrastructure_with_acc = infrastructure_with_acc.loc[infrastructure_with_acc['distance']<50]

In [29]:
import folium

def make_map(df):
    # Create a map object
    m = folium.Map(location=[48.8, 2.3], zoom_start=12)

    # Create the layer control
#     layer_control = folium.map.LayerControl(position='topright', collapsed=False)

    # Iterate through the rows of your dataset
    for i, row in df.iterrows():
        # Calculate the size of the marker based on the gravity of the accident
        size = row['grav'] * 10
        # Add a marker for the latitude and longitude
        accident = folium.RegularPolygonMarker(location=[row['lat'], row['long']], 
                                    number_of_sides=3, 
                                    radius=size,
                                    color='red', 
                                    fill_color='red')
        accident.add_to(m)
        #layer_control.add_child(accident, name="Accidents (proportional to gravity)")
        # Add a marker for the latitude_dep and longitude_dep
        infra1 = folium.CircleMarker(location=[row['latitude_dep'], row['longitude_dep']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra1.add_to(m)
        #layer_control.add_child(infra, name="Infrastructure (Cycling)")
        infra2 = folium.CircleMarker(location=[row['latitude_fin'], row['longitude_fin']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra2.add_to(m)
        # Draw a line between the latitude_dep, longitude_dep and latitude_fin, longitude_fin
        folium.PolyLine([[row['latitude_dep'], row['longitude_dep']], 
                         [row['latitude_fin'], row['longitude_fin']]], 
                        color='green', 
                        weight=2.5, 
                        opacity=1).add_to(m)

    # Add the layer control to the map
    #layer_control.add_to(m)
    # Show the map
    m.save("accident_with_infra.html")

In [ ]:
make_map(infrastructure_with_acc)

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
# imp = KNNImputer(n_neighbors=1)
# series_date = df["date"]
# df = df.drop('date', axis=1)
# df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
# # df = pd.concat([df, series_date])
# df

In [ ]:
# df = one_hot_encoder(df, ['lum', 'int', 'atm', 'col', 'catr', 'circ', 'vosp','prof', 'plan', 'surf', 'infra', 'situ', 'catv', 'obs', 'obsm', 'choc','grav', 'obs2', 'obsm2', 'choc2'])
# df

In [ ]:
# df["date"] = series_date
# df

In [ ]:
# print("After imputation")
# missing(df)

In [ ]:
# pca = PCA()
# df = df.drop('date', axis=1)
# X_reduced = pca.fit_transform(scale(df))
# X_reduced.shape

In [ ]:
# cumulative_explained_variance = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
# print(cumulative_explained_variance)

In [ ]:
# plt.bar(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ )
# plt.ylabel('Percentage of explained variance')
# plt.xlabel('Number of components')
# plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),
#          np.cumsum(pca.explained_variance_ratio_ ),
#          c='red',
#          label="Cumulative Explained Variance")
# plt.legend(loc='upper left')

In [ ]:
# inertie = []
# from tqdm import tqdm

# for i in tqdm(range(1, 300, 25)):
#     kmeans_pca = KMeans(n_clusters = i, init = 'k-means++', random_state=42)
#     kmeans_pca.fit(scale(df))
#     inertie.append(kmeans_pca.inertia_)

In [ ]:
# plt.figure(figsize = (10, 8))
# plt.plot(range(1,300, 25), inertie, marker = 'o', linestyle = '--')
# plt.xlabel('K-means with pca clustering with k cluster')
# plt.ylabel('Inertie')
# plt.show()

### Merge the infra_with_good_index and accident_with_infra_index

In [3]:
infra_with_good_index = pd.read_csv("../data/infra_with_good_index.csv")
accident_with_infra_index = pd.read_csv("../data/accident_with_infra_index.csv")

In [20]:
infra_with_good_index = infra_with_good_index.rename(columns={"Unnamed: 0":"index"})


In [21]:
infra_with_good_index

,index,id_local,id_osm,num_iti,code_com_d,ame_d,regime_d,sens_d,largeur_d,local_d,statut_d,revet_d,code_com_g,ame_g,regime_g,sens_g,largeur_g,local_g,statut_g,revet_g,access_ame,date_maj,trafic_vit,lumiere,d_service,source,project_c,ref_geo,geometry,latitude_dep,longitude_dep,latitude_fin,longitude_fin
0,0,geovelo_675971017_31142,675971017,NaN,31.0,BANDE CYCLABLE,NaN,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,LISSE,31.0,BANDE CYCLABLE,NaN,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,LISSE,NaN,2022-10-08,NaN,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.6028485 42.852490399999986, 0.60...",42.852490,0.602849,42.852617,0.602927
1,1,geovelo_675978309_31142,675978309,NaN,31.0,BANDE CYCLABLE,NaN,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,LISSE,31.0,AUCUN,NaN,NaN,NaN,NaN,EN SERVICE,LISSE,NaN,2022-10-08,NaN,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,LINESTRING (0.606002176828519 42.8649877380911...,42.864988,0.606002,42.864686,0.605712
2,2,geovelo_357472978_31542,357472978,V81:V83,31.0,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,NaN,31.0,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,NaN,NaN,2021-06-14,5.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.5927774 43.0603558, 0.5926679 43...",43.060356,0.592777,43.060316,0.592668
3,3,geovelo_955254797_31017,955254797,V83,31.0,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,RUGUEUX,31.0,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,RUGUEUX,VTC,2022-06-17,5.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.7080925 42.88769559999997, 0.710...",42.887696,0.708093,42.886258,0.710716
4,4,geovelo_95219324_31085,95219324,NaN,31.0,AUTRE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,NaN,31.0,AUTRE,AUTRE,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,NaN,VELO DE ROUTE,2020-09-21,5.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.7608928 42.92059970000001, 0.761...",42.920600,0.760893,42.920822,0.761309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81196,81196,geovelo_23032413_44049,23032413,NaN,44.0,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,LISSE,44.0,AUCUN,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,LISSE,NaN,2021-07-27,30.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (-2.5096905 47.2879328, -2.50975889...",47.287933,-2.509691,47.287990,-2.509759
81197,81197,geovelo_70576358_44049,70576358,NaN,44.0,AUCUN,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,RUGUEUX,44.0,DOUBLE SENS CYCLABLE NON MATERIALISE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,RUGUEUX,NaN,2021-07-28,30.0,True,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (-2.512557799999988 47.2950688, -2....",47.295069,-2.512558,47.294920,-2.512187
81198,81198,geovelo_967789927_44049,967789927,NaN,44.0,AUCUN,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,NaN,44.0,DOUBLE SENS CYCLABLE NON MATERIALISE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,NaN,NaN,2022-08-22,30.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (-2.5135449 47.29470449999999, -2.5...",47.294704,-2.513545,47.294504,-2.512892
81199,81199,geovelo_1052836530_44049,1052836530,NaN,44.0,BANDE CYCLABLE,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,CHAUSSEE,EN SERVICE,LISSE,44.0,AUCUN,EN AGGLOMERATION,UNIDIRECTIONNEL,NaN,NaN,EN SERVICE,LISSE,NaN,2022-04-18,50.0,NaN,NaN,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (-2.5212916 47.288513899999984, -2....",47.288514,-2.521292,47.288111,-2.521549


In [22]:
infrastructure_with_acc = accident_with_infra_index.merge(infra_with_good_index, left_on="infra_index", right_on="index")

In [24]:
infrastructure_with_acc.drop(columns=["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
import folium

def make_map(df):
    # Create a map object
    m = folium.Map(location=[48.8, 2.3], zoom_start=12)

    # Create the layer control
#     layer_control = folium.map.LayerControl(position='topright', collapsed=False)

    # Iterate through the rows of your dataset
    for i, row in df.iterrows():
        # Calculate the size of the marker based on the gravity of the accident
        size = row['grav'] * 10
        # Add a marker for the latitude and longitude
        accident = folium.CircleMarker(location=[row['lat'], row['long']], 
                                    color='red', 
                                    fill_color='red')
        accident.add_to(m)
        #layer_control.add_child(accident, name="Accidents (proportional to gravity)")
        # Add a marker for the latitude_dep and longitude_dep
        infra1 = folium.CircleMarker(location=[row['latitude_dep'], row['longitude_dep']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra1.add_to(m)
        #layer_control.add_child(infra, name="Infrastructure (Cycling)")
        infra2 = folium.CircleMarker(location=[row['latitude_fin'], row['longitude_fin']], 
                            radius=5, 
                            color='blue', 
                            fill=True, 
                            fill_color='blue')
        infra2.add_to(m)
        # Draw a line between the latitude_dep, longitude_dep and latitude_fin, longitude_fin
        folium.PolyLine([[row['latitude_dep'], row['longitude_dep']], 
                         [row['latitude_fin'], row['longitude_fin']]], 
                        color='green', 
                        weight=2.5, 
                        opacity=1).add_to(m)

    # Add the layer control to the map
    #layer_control.add_to(m)
    # Show the map
    display(m)